In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/276/proj

Mounted at /content/drive
/content/drive/My Drive/276/proj


In [ ]:
!pip install biopandas -q
!pip install dgl dglgo -f https://data.dgl.ai/wheels/repo.html -q
!sudo apt-get install pymol
!pip install graphein -q
!pip install graphein[extras]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.5/63.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.8/78.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 55.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed

In [ ]:
import requests
from biopandas.pdb import PandasPdb

import pandas as pd
from tqdm import tqdm

from graphein.protein.config import ProteinGraphConfig
from graphein.protein.graphs import construct_graph
from graphein.protein.visualisation import plotly_protein_structure_graph

[11/29/23 06:30:45] WARNING  To use the Graphein submodule                                         ]8;id=341570;file:///usr/local/lib/python3.10/dist-packages/graphein/protein/features/sequence/embeddings.py\embeddings.py]8;;\:]8;id=679462;file:///usr/local/lib/python3.10/dist-packages/graphein/protein/features/sequence/embeddings.py#44\44]8;;\
                             graphein.protein.features.sequence.embeddings, you need to install:                   
                             biovec                                                                                
                             To do so, use the following command: pip install biovec                               
                             Alternatively, you can install graphein with the extras:                              
                                                                                                                   
                             pip install graphein[extras]                                                          

                    WARNING  To use the Graphein submodule graphein.protein.visualisation, you  ]8;id=813522;file:///usr/local/lib/python3.10/dist-packages/graphein/protein/visualisation.py\visualisation.py]8;;\:]8;id=336162;file:///usr/local/lib/python3.10/dist-packages/graphein/protein/visualisation.py#35\35]8;;\
                             need to install: pytorch3d                                                            
                             pytorch3d cannot be installed via pip                                                 

                    WARNING  To use the Graphein submodule graphein.protein.meshes, you need to        ]8;id=752279;file:///usr/local/lib/python3.10/dist-packages/graphein/protein/meshes.py\meshes.py]8;;\:]8;id=986275;file:///usr/local/lib/python3.10/dist-packages/graphein/protein/meshes.py#29\29]8;;\
                             install: pytorch3d                                                                    
                             To do so, use the following command: pip install pytorch3d                            

In [ ]:
def download_and_save_pdb(pdb_id, file_path):
  """ Download a PDB file using its ID and save it to a specified path. """
  url = f"https://files.rcsb.org/download/{pdb_id}.pdb"
  response = requests.get(url)
  if response.status_code == 200:
    with open(file_path, 'w') as file:
      file.write(response.text)
    return True  # Return True to indicate successful download
  else:
    print(f"Warning: Failed to download PDB file with ID {pdb_id}")
    return False  # Return False to indicate a failed download

# Example usage
df = pd.read_csv("Biodata.csv")

successful_downloads = 0  # Counter for successful downloads

# Adding a progress bar using tqdm
for pdb_id in tqdm(df.structureId.values, desc="Downloading PDB files"):
  try:
    file_path = "/content/drive/My Drive/proj/tmp/" + f"{pdb_id}.pdb"
    if download_and_save_pdb(pdb_id, file_path):
      successful_downloads += 1
  except Exception as e:
    print(f"An error occurred with PDB ID {pdb_id}: {e}")

print(f"Total successful downloads: {successful_downloads}")

Total successful downloads: 4996


In [ ]:
from graphein.protein.config import ProteinGraphConfig
from graphein.protein.graphs import construct_graph

# Load the default config
c = ProteinGraphConfig(granularity='CA')

# Construct the graph!
g = construct_graph(path=file_path)

Output()

/usr/local/lib/python3.10/dist-packages/graphein/protein/graphs.py:161: FutureWarning:

The default value of regex will change from True to False in a future version.

/usr/local/lib/python3.10/dist-packages/graphein/protein/graphs.py:164: FutureWarning:

The default value of regex will change from True to False in a future version.

[11/27/23 00:56:19] DEBUG    Deprotonating protein. This removes H atoms from the pdb_df dataframe    ]8;id=550750;file:///usr/local/lib/python3.10/dist-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=163055;file:///usr/local/lib/python3.10/dist-packages/graphein/protein/graphs.py#187\187]8;;\

                    DEBUG    Detected 154 total nodes                                                 ]8;id=48785;file:///usr/local/lib/python3.10/dist-packages/graphein/protein/graphs.py\graphs.py]8;;\:]8;id=486519;file:///usr/local/lib/python3.10/dist-packages/graphein/protein/graphs.py#431\431]8;;\

In [ ]:
p = plotly_protein_structure_graph(
    g,
    colour_edges_by="kind",
    colour_nodes_by="degree",
    label_node_ids=True,
    plot_title="Peptide backbone graph. Nodes coloured by degree.",
    node_size_multiplier=1
    )
p.show()